In [1]:
import pandas as pd
from pathlib import Path
import fuzzymatcher
import recordlinkage
import numpy as np
import time
import math

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# 1. Load Data

In [2]:
left_table = pd.read_csv('ltable.csv', encoding = 'latin1')
right_table = pd.read_csv('rtable.csv', encoding = 'latin1')

In [3]:
left_table.head()

,ltable_id,title,category,brand,modelno,price
0,0,draper infrared remote transmitter,electronics - general,draper,121066,58.45
1,1,epson 1500 hours 200w uhe projector lamp elplp12,monitors,epson,elplp12,438.84
2,2,comprehensive two-piece 75 precision bnc jack ...,tv accessories,comprehensive,bj-2c7559,59.25
3,3,d-link dcs-1100 network camera,garden - general,d-link,dcs-1100,99.82
4,4,startech.com rkpw247015 24 outlet power strip,electronics - general,startech,rkpw247015,59.00


In [4]:
right_table.head()

,rtable_id,title,category,brand,modelno,price
0,0,koss eq50 3-band stereo equalizer,headphone accessories,koss,152132,12.65
1,1,kodak black ink cartridge 10b 1163641,inkjet printer ink,kodak,1163641,10.28
2,2,kingston 128mx64 pc2700 compaq evo d320 ktc-d3...,computers accessories,kingston,ktc-d320 / 1g,33.75
3,3,kinamax ms-ues2 mini high precision usb 3-butt...,mice,kinamax,ms-ues2,6.99
4,4,kensington k72349us wireless mouse for netbooks,mice,kensington,k72349us,24.00


# 2. Entity Matching

**Create record linkage, merging on `title`**

In [5]:
indexer = recordlinkage.Index()
indexer.sortedneighbourhood(['modelno'])
candidates = indexer.index(left_table, right_table)
print(len(candidates))

4314


**Create record linkage comparison**

**I tuned the threshold parameter to optimize the f1 score**

In [6]:
%%time

threshold = 0.95

compare = recordlinkage.Compare()
compare.string('title',
               'title',
               threshold=threshold,
               label='title')

compare.string('category',
               'category',
               threshold=threshold,
               label='category')

compare.string('brand',
               'brand',
               threshold=threshold,
               label='brand')

compare.string('modelno',
               'modelno',
               threshold=threshold, 
               label='modelno')

compare.numeric('price',
               'price',
               label='price')

features = compare.compute(candidates, 
                           left_table,
                           right_table)

CPU times: user 338 ms, sys: 8.21 ms, total: 346 ms
Wall time: 360 ms


In [7]:
features.apply(pd.Series.value_counts).dropna()

,title,category,brand,modelno,price
0.0,4286.0,4188.0,2421.0,3672.0,3910
1.0,28.0,126.0,1893.0,642.0,70


**Create score based on summing rows**

In [8]:
NUM_FEATURES = len(features.columns)
features["score"] = features.sum(axis=1)

In [9]:
features = (features.reset_index()
                   .rename(columns={"level_0":"ltable_id",
                                    "level_1":"rtable_id"}))
features.head()

,ltable_id,rtable_id,title,category,brand,modelno,price,score
0,0,18637,0.0,0.0,1.0,0.0,0.0,1.0
1,1,13161,0.0,0.0,1.0,0.0,0.0,1.0
2,2,1808,0.0,0.0,0.0,0.0,0.0,0.0
3,3,4377,0.0,0.0,1.0,0.0,0.0,1.0
4,4,21625,0.0,0.0,0.0,0.0,0.0,0.0


**Predict if IDs are the same if floor(NUM_FEATURES/2) columns are == 1**

In [10]:
features['prediction'] = [0]*len(features)
features.loc[features.score >= math.floor(NUM_FEATURES/2), 'prediction'] = 1

In [11]:
features["prediction"].value_counts()

0    3701
1     613
Name: prediction, dtype: int64

In [12]:
out_frame = features[['ltable_id', 'rtable_id', 'prediction']]
out_frame.to_csv('output.csv')

# 3. Prediction

**Load data**

In [13]:
prediction = pd.read_csv('output.csv', encoding = 'latin1')
truth = pd.read_csv('test.csv', encoding = 'latin1')
print("Prediction columns: ", list(prediction.columns))
print("Truth columns: ", list(truth.columns))

Prediction columns:  ['Unnamed: 0', 'ltable_id', 'rtable_id', 'prediction']
Truth columns:  ['ltable_id', 'rtable_id', 'label', 'id']


**Merge (inner) prediction DF on truth DF to get same set of IDs**

In [14]:
filt_prediction = (prediction.merge(truth, 
                            how="inner",
                            on=["ltable_id", "rtable_id"])
                            .rename(columns={"label":"y_true",
                                             "prediction":"y_pred"})
                            [["ltable_id","rtable_id","y_pred","y_true"]])
filt_prediction.head()

,ltable_id,rtable_id,y_pred,y_true
0,16,15183,0,0
1,21,8915,0,0
2,29,8070,0,0
3,50,2013,0,0
4,61,11091,0,0


In [15]:
y_true = list(filt_prediction["y_true"])
y_pred = list(filt_prediction["y_pred"])

**Display f1 score, precision, and recall, false-positive, and false-negative**

In [16]:
print("Confusion Matrix with FP (upper right) and FN (lower left)")
confusion_matrix(y_true, y_pred)

## TP: upper left
## TN: lower right
## FP: upper right
## FN: lower left

Confusion Matrix with FP (upper right) and FN (lower left)


array([[342,  23],
       [ 66, 267]])

In [17]:
print("F1 score:", round(f1_score(y_true, y_pred),4))

F1 score: 0.8571


In [18]:
print("Precision:", round(precision_score(y_true, y_pred),4))

Precision: 0.9207


In [19]:
print("Recall:", round(recall_score(y_true, y_pred),4))

Recall: 0.8018


# 4. Summary

- In order to be considered a match, I decided the score must be 3 or higher. Of the columns `product name`, `category`, `brand`, `modelno`, and `price`, I decided if product name, brand, and model number match, it is very likely the products are a true match. Though category and price are important, these attributes can be listed differently even if the product is the same. 

- I knew index blocking would be useful to minimize time and add a filter from the start. I chose to use the the SortedNeighborhood algorithm to minimize false negatives. If there are spelling mistakes in the data, soley blocking would fail because it would exclude potential matches based on these minor spelling mistakes. I used model number as the primary attribute for the index because it is the most specific and unique identifyer of a given product. Although brand and title are also idicative of a match, they could give many false positives since brands carry many similar products. 

- For the secondary attributes of the comparison, I used category, brand, model number, and price. I considered not includng price because I know that the same product can be priced differently from store to store. However, in tuning the parameters I found including the price resulted in the highest F1 score. 

- Another parameter I tuned was the threshold for each comparision. String comparisons greater than or equal to this threshold parameter are 1, otherwise 0. In class we used 0.85, but I wondered if increasing this threshold would be benifical. I fonud that using a threshold of 0.95 resulted in the highest F1 measure.

- I additionally set that products are considered a match (prediction = 1) if score >= num_features/2 because this enforces that at least half of the features match. 